**Ejercicio 1:** Suponga que un viajante tiene que visitar n ciudades en el menor tiempo posible. Considere una matriz D de tama ̃no n×n cuyos elementos dpq denotan la distancia entre cada par de ciudades (p,q). Se define un recorrido como una trayectoria cerrada que visita cada ciudad una y s ́olo una vez (a excepci ́on de la ciudad de partida, a la cual debe regresar). El problema consiste
en hallar el recorrido de m ́ınima longitud.

Implemente el algoritmo de sistema de hormigas y util ́ıcelo para resolver el problema del agente viajero considerando los datos proporcionados en el archivo gr17.csv. Analice el efecto de la tasa de evaporación (ρ) y de la cantidad de feromona depositada (τ) sobre los resultados de la b ́usqueda. Para esto  ́ultimo compare el desempe ̃no del algoritmo empleando los m ́etodos global, local y uniforme
para dep ́osito de feromonas. Realice varias corridas con cada configuraci ́on experimental y considere el tiempo de b ́usqueda y la longitud de los caminos encontrados como medidas para comparar el desempe ̃no. Construya una tabla comparativa con los resultados obtenidos.

In [233]:
import pandas as pd
import numpy as np
import random
import time

In [234]:
def select_next_city(current_city, unvisited_cities, pheromone_matrix, distance_matrix, alpha, beta):
    prob_of_next = np.zeros_like(unvisited_cities, dtype=float)                                 # Genero un vector de probabilidades relleno de zeros
    total_sum = 0                                                                               # Genero una variable para almacenar la sumatoria
    for i in range(len(unvisited_cities)):                                                      # Por cada ciudad sin visitar 
        unvisited_idx = unvisited_cities[i]                                                         # Determino el indice de la ciudad 
        total_sum += (((pheromone_matrix[current_city][unvisited_idx])**alpha)*                     # Calculo la suma segun distancias y feromonas
                      ((1/distance_matrix[current_city][unvisited_idx])**beta))
    for i in range(len(unvisited_cities)):                                                      # Por cada ciudad sin visitar
        unvisited_idx = unvisited_cities[i]                                                         # Determino el indice de la ciudad
        prob_of_next[i] = (((pheromone_matrix[current_city][unvisited_idx])**alpha)*                # Calculo la probabilidad segun distancias y feromonas
                           ((1/distance_matrix[current_city][unvisited_idx])**beta))/total_sum
    total_prob = sum(prob_of_next)                                                              # Sumo todas las probabilidades
    normalized_prob = [p / total_prob for p in prob_of_next]                                    # Las normalizo entre 0 y 1
    random_number = random.random()                                                             # Genero un valor random en ese rango
    cumul_prob = 0                                                                              # Inicializo la probabilidad acumulada
    for i, prob in enumerate(normalized_prob):                                                  # Para cada probabilidad
        cumul_prob += prob                                                                          # Agrego la probabilidad a la acumulada
        if (random_number <= cumul_prob):                                                           # Si el valor random es menor que la probabilidad acumulada
            return unvisited_cities[i]                                                                  # Retorno la ciudad ganadora

In [235]:
def deposit_pheromones_Qcte(n_ants, n_cities, pheromone_matrix, tour_matrix, Qfer, _):
    for i in range(n_ants):                                                                 # Por cada viajante
        delta_pheromones = np.zeros_like(pheromone_matrix)                                      # Inicializacion de la matriz de cambio en feromonas
        for j in range(-1, n_cities-1):                                                         # Por cada camino
            idx_act_city  = int(tour_matrix[i][j])                                                  # Indice ciudad actual
            idx_next_city = int(tour_matrix[i][j+1])                                                # Indice ciudad siguiente
            delta_pheromones[idx_act_city][idx_next_city] += Qfer                                      # Actualización por Q.
            delta_pheromones[idx_next_city][idx_act_city] += Qfer                                      # Arranca en -1 para que tambíen cubra la vuelta a la ciudad de origen
        pheromone_matrix += delta_pheromones                                                    # Depósito de nuevas feromonas
    return pheromone_matrix

In [236]:
def deposit_pheromones_Qvar(n_ants, n_cities, pheromone_matrix, tour_matrix, Qfer, distance_matrix):
    for i in range(n_ants):                                                                                         # Por cada viajante
        delta_pheromones = np.zeros_like(pheromone_matrix)                                                          # Inicializacion de la matriz de cambio en feromonas
        for j in range(-1, n_cities-1):                                                                             # Por cada camino
            idx_act_city  = int(tour_matrix[i][j])                                                                  # Indice ciudad actual
            idx_next_city = int(tour_matrix[i][j+1])                                                                # Indice ciudad siguiente
            delta_pheromones[idx_act_city][idx_next_city] += Qfer/distance_matrix[idx_act_city][idx_next_city]      # Actualización por Q/dist.
            delta_pheromones[idx_next_city][idx_act_city] += Qfer/distance_matrix[idx_next_city][idx_act_city]      # Arranca en -1 para que tambíen cubra la vuelta a la ciudad de origen
        pheromone_matrix += delta_pheromones                                                                        # Depósito de nuevas feromonas
    return pheromone_matrix

In [237]:
def deposit_pheromones_Qltot(n_ants, n_cities, pheromone_matrix, tour_matrix, Qfer, distance_matrix):
    for i in range(n_ants):
        total_dist = 0                                                         
        delta_pheromones = np.zeros_like(pheromone_matrix)                          # Inicializacion de la matriz de cambio en feromonas
        for j in range(-1, n_cities-1):                                             # Calculo distancia total recorrida por la hormiga 
            idx_act_city  = int(tour_matrix[i][j])                                  # Indice ciudad actual
            idx_next_city = int(tour_matrix[i][j+1])                                # Indice ciudad siguiente
            total_dist += distance_matrix[idx_act_city][idx_next_city]
        for j in range(-1, n_cities-1):                                             # Por cada par (i,j), calculo cambio en feromonas                 
            delta_pheromones[idx_act_city][idx_next_city] += Qfer/total_dist        # Actualización por Q/total_dist
            delta_pheromones[idx_next_city][idx_act_city] += Qfer/total_dist        # Arranca en -1 para que tambíen cubra la vuelta a la ciudad de origen
        pheromone_matrix += delta_pheromones                                        # Depósito de nuevas feromonas
    return pheromone_matrix

In [238]:
# Función que obtinene la distancia mínima entre las n hormigas
def min_distance(tour_matrix, distance_matrix):          
    min_dist = float('inf')

    for k in range(tour_matrix.shape[0]):  # Por cada hormiga
        dist_k = 0
        for i in range(len(tour_matrix[k]) - 1):
            dist_k += distance_matrix[tour_matrix[k][i]][tour_matrix[k][i+1]]
        dist_k += distance_matrix[tour_matrix[k][-1]][tour_matrix[k][0]]  # Volver a la ciudad de inicio
        
        if(dist_k < min_dist):
            idx_min_dist = k
            min_dist = dist_k

    return tour_matrix[idx_min_dist],min_dist 

In [239]:
#--- Carga de datos ---#
distance_matrix = np.genfromtxt('./data/gr17.csv', delimiter=',')   # Matriz de distancias
pheromone_matrix = np.ones_like(distance_matrix)                    # Matriz de feromonas (inicializada llena de 1s) 

#--- Inicialización de parámetros ---#
n_ants = 30                            # Número de hormigas
max_it = 100                           # Número de iteraciones
evap_var = 0.3                         # Tasa de evaporación de feromonas
Qfer = 1                               # Cantidad de feromonas a depositar
alpha = 1                              # Exponente para feromona
beta = 1                               # Exponente para distancia
best_tour = [None, float('inf')]       # Mejor camino conseguido
n_cities = distance_matrix.shape[0]    # Longitud del camino total

In [240]:
deposit_func = [deposit_pheromones_Qcte, deposit_pheromones_Qvar, deposit_pheromones_Qltot]
for k in range(3):
    best_vec = []
    best_dis = np.inf
    inicio = time.time()

    for _ in range(max_it):                                                                      # Por cada época
        tour_matrix = np.zeros((n_ants, n_cities), dtype=int)                                        # Determino una estructura para guardar los tours
        for i in range(n_ants):                                                                      # Por cada viajante
            current_city = 0                                                                             # Determino la ciudad inicial
            unvisited_cities = np.arange(current_city+1, len(distance_matrix))                           # Genero un vector de ciudades no visitadas
            #print(unvisited_cities)         
            for j in range(n_cities-1):                                                                  # Por cada ciudad sin visitar
                next_city = select_next_city(current_city, unvisited_cities, pheromone_matrix,             # Determino la próxima ciudad del viajante
                                            distance_matrix, alpha, beta)                  
                current_city = next_city                                                                   # Avanzo hacia la siguiente ciudad
                tour_matrix[i,j+1] = int(next_city)                                                          # Agrego la ciudad de la lista de visitadas   
                unvisited_cities = np.delete(unvisited_cities,                                               # Elimino la ciudad de la lista por visitar  
                                np.where(unvisited_cities == next_city))             
                #print(unvisited_cities)            
        pheromone_matrix *= (1-evap_var)                                                            # Evaporación de feromonas anteriores

        pheromone_matrix = deposit_func[k](n_ants, n_cities, pheromone_matrix, tour_matrix, Qfer, distance_matrix)     # Depósito de nuevas feromonas
        
        vec, distance = min_distance(tour_matrix, distance_matrix)
        if(best_dis > distance):    # Si distancia mín entre las hormigas para esta iteracion es la distancia mín en historia total, reemplazo
            best_dis = distance
            best_vec = vec
    fin = time.time()
    print(deposit_func[k].__name__,' - ',best_vec,' - ',best_dis, ' - ',fin-inicio,'seg.')
    
    #----------------------- Best distance: 2085 -----------------------#
    #------ [ 0  3 12  6  7  5 16 13 14  2 10  9  1  4  8 11 15 ] -------# 
    #-------------------------------------------------------------------#


deposit_pheromones_Qcte  -  [ 0  3 12  6  7  5 16 13 14  2 10  9  1  4  8 11 15]  -  2085.0  -  2.3349826335906982 seg.
deposit_pheromones_Qvar  -  [ 0  3 12  6  7  5 16 13 14  2 10  9  1  4  8 11 15]  -  2085.0  -  2.3817567825317383 seg.
deposit_pheromones_Qltot  -  [ 0 12  3  6  7  5 16 13 14  2 10  4  1  9  8 11 15]  -  2158.0  -  2.547287702560425 seg.
